In [ ]:
# Execute this cell and reassure you're using python 3.9. Swich to 3.9 otherwise using "Command palette" in the right bottom corner of Colab.
!python --version

# EXECUTE THE CELL BELOW ONLY IF YOU'RE WORKING IN GOOGLE COLAB! 

In [ ]:
# Execute the cell only ONCE

!git clone https://gitflic.ru/project/aidynamicaction/classedu2023-advctrl.git
%cd /content/classedu2023-advctrl/assignments/asgn-3

# Introduction
In this assignment, you will learn how to design a controller based on backstepping stabilization technique. For this purpose you're intended to

* first, implement an F-disassembled CLF controller for kinematic robot
* second, implement its supersystem - dynamical variant of 3-wheeled robot
* third, implement a backstepping controller for a dynamic 3-wheeled robot

**It's strongly recommended for you to familiarize yourself with [README](../README.md) in `assignments` folder before starting to work on your tasks.**

In [1]:
# Execute the cell only ONCE
# Restart runtime RIGHT AFTER the cell is finished. It may fail with error --- do not mind
%pip install -e src

Obtaining file:///C:/Users/serge/Desktop/classedu2023-advctrl/assignments/asgn-3/src
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for rcognita (pyproject.toml): started
  Building editable for rcognita (pyproject.toml): finished with status 'done'
  Created wheel for rcognita: filename=rcognita-0.0.0-0.editable-py3-none-any.whl size=2683 sha256=6492c4e145b52d274c53a89d3cc08f1c7aa9bbe505e28bdb90b736d8484eeafb
  Stored in directory: C:\Users\serge\AppData\Local\Temp\pip-ephem-wheel-cache-m0e_9tci\wheels\6d\cd\1c\a58d88453b569

In [2]:
exit()

: 

In [1]:
from rcognita.__utilities import rc, Clock
import rcognita.__utilities as utilities
from rcognita.systems import System, Sys3WRobotNI
from rcognita.controllers import Controller
from rcognita.testing_pipeline import create_testing_pipeline
from rcognita.visualization import dashboards
from rcognita.models import ModelQuadForm
import rcognita
import numpy as np
from matplotlib import rc as rc_params
from scipy.optimize import minimize, Bounds
rc_params("animation", html="jshtml")


c:\Users\serge\Desktop\classedu2023-advctrl\assignments\asgn-3\asgn_3_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def apply_action_bounds(method):
    def wrapper(self, *args, **kwargs):
        self.action = method(self, *args, **kwargs)
        if hasattr(self, "action_bounds") and self.action_bounds != []:
            action = np.clip(
                self.action, self.action_bounds[:, 0], self.action_bounds[:, 1]
            )
            self.action = action
        return self.action

    return wrapper

## Task 1: F-disassembled CLF controller

---------
<img style="right;" src="3wrobot.svg" width=18% height=18%>

Let's introduce kinematic robot (a.k.a. nonholonomic integrator or NI) and it's supersystem - dynamic robot (a.k.a. extended nonholonomic double integrator or ENDI).

### Kinematic robot (NI)

NI is represented by the following system:

\begin{cases}
\dot{x}=v \cos \theta \\
\dot{y}=v \sin \theta \\
\dot{\alpha}=\omega
\end{cases}

Which can be rewritten (as we discussed in the 1-st assignment) in non-holonomic coordinates as

\begin{cases}
\dot{x}_1=\eta_1 \\
\dot{x}_2=\eta_2 \\
\dot{x}_3=x_2 \eta_1-x_1 \eta_2
\end{cases}

the final form of which is 
$
\dot{x}=f_{\mathrm{NI}}(x, u)=\underbrace{\left(\begin{array}{c}
1 \\
0 \\
-x_2
\end{array}\right)}_{=: g_1(x)} u_1+\underbrace{\left(\begin{array}{c}
0 \\
1 \\
x_1
\end{array}\right)}_{=: g_2(x)} u_2 $

### Dynamic robot (ENDI)

ENDI is represented by an extended version of previous system:

\begin{cases}
\dot{x} & =v \cos \theta \\
\dot{y} & =v \sin \theta \\
\dot{\alpha} & =\omega \\
\dot{v} & =\frac{1}{m} F \\
\dot{\omega} & =\frac{1}{I} M
\end{cases}

and has the corresponding representation in non-holonomic coordinates:

\begin{cases}
\dot{x}_1 & =\eta_1 \\
\dot{x}_2 & =\eta_2 \\
\dot{x}_3 & =x_2 \eta_1-x_1 \eta_2 \\
\dot{\eta}_1 & =u_1 \\
\dot{\eta}_2 & =u_2
\end{cases}

### Backstepping approach
The whole workaround with backstepping can be boiled down into two major parts:

1. Derive stabilizing policy for subsystem (NI in our case)
2. Use the stabilizing policy for subsystem to derive a stabilizing policy for it's supersystem (ENDI in our case)

Let's start from the first part.

### Optimization-based controller for NI

First of all, consider the following function:


$F(x ; \theta):=x_1^4+x_2^4+\frac{\left|x_3\right|^3}{\left(x_1 \cos (\theta)+x_2 \sin (\theta)+\sqrt{\left|x_3\right|}\right)^2}$

Then we can write out a CLF as:

$V(x):=\min _{\theta \in \Theta} F(x ; \theta)$, where $F(x ; \theta)$ is an $F$-disassembled CLF for (NI). 

Choose a minimizer $\theta^{\star} \in \Theta^{\star}:=\arg \min _{\theta \in \Theta} F(x ; \theta)$ and compute $\zeta(x ; \theta)=\nabla_x F(x ; \theta)$. Then, $\zeta\left(x ; \theta^{\star}\right) \in$ $\partial_D^F V(x)=\left.\nabla_x F(x ; \theta)\right|_{\theta \in \Theta^*}$ holds.

Use the following formula to derive a stabilizing policy:
$$
\kappa(x ; \theta^{\star})=-\left(\begin{array}{c}
\left\langle\zeta(x ; \theta^{\star}), g_1(x)\right\rangle \\
\left\langle\zeta(x ; \theta^{\star}), g_2(x)\right\rangle
\end{array}\right),
$$

Where $\left[g_1(x) \quad g_2(x)\right]=\left(\begin{array}{cc}1 & 0 \\ 0 & 1 \\ x_2 & -x_1\end{array}\right)$


So, your task here is to implement the following methods:

1. 
    ```python
    def _F(self, xNI, theta)
    ```
    which is an implementation of $F(x ; \theta)$

2. 
    ```python
    def _zeta(self, xNI)
    ```
    which is a method implementing $\zeta(x ; \theta^{\star})$ (You can use scipy optimizer to get a corresponding minimizer - check imports)

3.
    ```python
    def _kappa(self, xNI)
    ```
    which is a method computing stabilizing policy $\kappa(x ; \theta^{\star})$ in non-holonomic coordinates

In [1]:
class Controller3WRobotNIDisassembledCLF:
    def __init__(
        self, controller_gain=10, action_bounds=None, time_start=0, sampling_time=0.1
    ):
        self.controller_gain = controller_gain
        self.action_bounds = action_bounds
        self.controller_clock = time_start
        self.time_start = time_start
        self.sampling_time = sampling_time
        self.Ls = []
        self.times = []
        self.action_old = rc.zeros(2)
        self.clock = Clock(period=sampling_time, time_start=time_start)

    def reset(self):
        """
        Resets controller for use in multi-episode simulation.

        """
        self.controller_clock = self.time_start
        self.action_old = rc.zeros(2)

    def _F(self, xNI, theta):
        """
        Marginal function for NI.

        """

        ####################
        # YOUR CODE GOES HERE
        ####################

        x_1 = xNI[0]
        x_2 = xNI[1]
        x_3 = xNI[2]

        F = (
            np.power(x_1, 4)
            + np.power(x_2, 4)
            + np.divide(
                (np.power(np.abs(x_3),3))
                , (
                    np.power(
                        (
                            x_1 * np.cos(theta)
                            + x_2 * np.sin(theta)
                            + np.sqrt(np.abs(x_3))
                        ),
                        2
                    )
                )
            )
        )

        ####################
        # YOUR CODE ENDS HERE
        ####################

        return F

    def _zeta(self, xNI):
        """
        Disassembled supper_bound_constraint gradient, with finding minimizer theta.

        """

        ####################
        # YOUR CODE GOES HERE
        ####################

        
        theta=0
        theta_star=minimize(lambda self._F: self._F(xNI,theta), x0=0, args=theta, solver="trustconstr")
        grad=
        
       
        ####################
        # YOUR CODE ENDS HERE
        ####################
        return grad

    def _kappa(self, xNI):
        """
        Stabilizing controller for NI-part.
        """
        kappa_val = rc.zeros(2)
        G = rc.zeros([3, 2])

       
        ####################
        # YOUR CODE GOES HERE
        ####################

        x_1 = xNI[0]
        x_2 = xNI[1]

        G=([    [    1,      0       ],
                [    0,      1       ],
                [    x_2,    -x_1    ]   ])
        
        kappa_val= -([[],
                      []])

        ####################
        # YOUR CODE ENDS HERE
        ####################

        return kappa_val

    def _Cart2NH(self, coords_Cart):
        """
        Transformation from Cartesian coordinates to non-holonomic (NH) coordinates.

        """

        xNI = rc.zeros(3)

        xc = coords_Cart[0]
        yc = coords_Cart[1]
        angle = coords_Cart[2]

        xNI[0] = angle
        xNI[1] = xc * rc.cos(angle) + yc * rc.sin(angle)
        xNI[2] = -2 * (yc * rc.cos(angle) - xc * rc.sin(angle)) - angle * (
            xc * rc.cos(angle) + yc * rc.sin(angle)
        )

        return xNI

    def _NH2ctrl_Cart(self, xNI, uNI):
        """
        Get control for Cartesian NI from NH coordinates.

        """

        uCart = rc.zeros(2)

        uCart[0] = uNI[1] + 1 / 2 * uNI[0] * (xNI[2] + xNI[0] * xNI[1])
        uCart[1] = uNI[0]

        return uCart

    @apply_action_bounds
    def compute_action_sampled(self, time, state, observation, observation_target=[]):
        """
        Compute sampled action.

        """

        is_time_for_new_sample = self.clock.check_time(time)

        if is_time_for_new_sample:  # New sample
            action = self.compute_action(state, observation)
            self.times.append(time)
            self.action_old = action

            return action

        else:
            return self.action_old

    def compute_action(self, state, observation, time=0, observation_target=[]):
        """
        Same as :func:`~Controller3WRobotNIDisassembledCLF.compute_action`, but without invoking the internal clock.

        """

        xNI = self._Cart2NH(observation)
        kappa_val = self._kappa(xNI)
        uNI = self.controller_gain * kappa_val
        self.action = self._NH2ctrl_Cart(xNI, uNI)

        self.action_old = self.action

        return self.action

SyntaxError: invalid syntax (645268737.py, line 38)

# Testing

In [ ]:
state_init = np.array([5, 5, 3 * np.pi / 4])
action_init = np.ones(2)

system = Sys3WRobotNI(
    dim_input=2,
    dim_output=3,
    sys_type="diff_eqn",
    dim_state=3,
    dim_disturb=3,
    pars=[]
)
controller = Controller3WRobotNIDisassembledCLF(
    controller_gain=10, action_bounds=np.array([[-25, 25],[-5, 5]]), time_start=0, sampling_time=0.01
)

dashboard = dashboards.RobotTrackingDasboard

scenario, animator = create_testing_pipeline(
    system, 
    controller, 
    state_init=state_init, 
    action_init=action_init, 
    model_of_running_objective = ModelQuadForm(weights=np.diag([5, 5, 1, 0, 0])),
    observation_namings=["x", "y", "angle"],
    action_namings=["lin. speed", "ang. speed"],
    system_visualization_dashboard=dashboard,
    system_visualization_dashboard_params={
        "time_start": 0, 
        "x_max": 10, 
        "x_min": -10, 
        "y_max": 10, 
        "y_min": -10
        },
    time_final=15,
    )

### Here you can debug your code. When the code is changed, rerun previous cells to update the scenario cache.

In [ ]:
scenario.run() # <--- takes some time. (In our case about 30 seconds)
trajectory = []
for i , (k, v) in enumerate(scenario.cache.items()):
    trajectory.append(v[3])

trajectory[-30:]

### Run the animation to finalize your results

In [ ]:
%%capture
animation = animator.get_animation()

In [ ]:
animation # <--- 1-2 minutes to render

## Dynamical 3-wheeled robot

Implement the following system

---------

$$
\begin{cases}
\dot{x} & =v \cos \theta \\
\dot{y} & =v \sin \theta \\
\dot{\alpha} & =\omega \\
\dot{v} & =\frac{1}{m} F \\
\dot{\omega} & =\frac{1}{I} M
\end{cases}
\text{
where 
$x_{\text {Cart }}:=(x, y, \alpha, v, \omega)^{\top}$ 
and $u_{\text {Cart }}:=(F, M)^{\top}$}
$$

In [ ]:
class Sys3WRobot(System):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.name = "3wrobot"

    def compute_dynamics(self, time, state, action, disturb=None):
        Dstate = utilities.rc.zeros(
            self.dim_state,
            prototype=(state, action),
        )

        m, I = self.pars[0], self.pars[1]

        ####################
        #YOUR CODE GOES HERE
        ####################

        ####################
        #YOUR CODE ENDS HERE
        ####################

        return Dstate

    def out(self, state, time=None, action=None):
        return state

### Task 2: Backstepping-based Controller

So, your task here is to implement the following methods:

1. 
    ```python
    def _Fc(self, xNI, theta) # Augmented F
    ```
    which is an implementation of $Fc(x ; \theta)$

2. 
    ```python
    def _zeta(self, xNI) # Nabla F_c
    ```
    Copy-paste from previous task

3.
    ```python
    def _kappa(self, xNI)
    ```
    Copy-paste from previous task

In [ ]:
class Controller3WRobotDisassembledCLF:

    def __init__(
        self,
        m,
        I,
        controller_gain=100,
        action_bounds=None,
        time_start=0,
        sampling_time=0.01,
    ):
        self.m = m
        self.I = I
        self.controller_gain = controller_gain
        self.action_bounds = action_bounds
        self.controller_clock = time_start
        self.sampling_time = sampling_time
        self.clock = Clock(period=sampling_time, time_start=time_start)

        self.action_old = rc.zeros(2)


    def reset(self):
        self.action_old = rc.zeros(2)

    def _zeta(self, xNI, theta):

        ####################
        #YOUR CODE GOES HERE
        ####################

        ####################
        #YOUR CODE ENDS HERE
        ####################
  

        return grad

    def _kappa(self, xNI, theta):
        """
        Stabilizing controller for NI-part.

        """

        G = rc.zeros([3, 2])

        kappa_val = rc.zeros(2, prototype=theta)

        ####################
        #YOUR CODE GOES HERE
        ####################

        ####################
        #YOUR CODE ENDS HERE
        ####################

        return kappa_val

    def _Fc(self, xNI, eta, theta):

        ####################
        #YOUR CODE GOES HERE
        ####################

        ####################
        #YOUR CODE ENDS HERE
        ####################

        return Fc_value

    def _Cart2NH(self, coords_Cart):

        xNI = rc.zeros(3)
        eta = rc.zeros(2)

        xc = coords_Cart[0]
        yc = coords_Cart[1]
        angle = coords_Cart[2]
        v = coords_Cart[3]
        omega = coords_Cart[4]

        xNI[0] = angle
        xNI[1] = xc * rc.cos(angle) + yc * rc.sin(angle)
        xNI[2] = -2 * (yc * rc.cos(angle) - xc * rc.sin(angle)) - angle * (
            xc * rc.cos(angle) + yc * rc.sin(angle)
        )

        eta[0] = omega
        eta[1] = (yc * rc.cos(angle) - xc * rc.sin(angle)) * omega + v

        return [xNI, eta]

    def _NH2ctrl_Cart(self, xNI, eta, uNI):

        uCart = rc.zeros(2)

        uCart[0] = self.m * (
            uNI[1]
            + xNI[1] * eta[0] ** 2
            + 1 / 2 * (xNI[0] * xNI[1] * uNI[0] + uNI[0] * xNI[2])
        )
        uCart[1] = self.I * uNI[0]

        return uCart

    def compute_action_sampled(self, time, state, observation, observation_target=[]):
        is_time_for_new_sample = self.clock.check_time(time)

        if is_time_for_new_sample:
            self.action = self.compute_action(time, state, observation)

            self.action_old = self.action

            return self.action

        else:
            return self.action_old

    @apply_action_bounds
    def compute_action(self, time, state, observation, observation_target=[]):

        xNI, eta = self._Cart2NH(observation)
        kappa_val = self._kappa(xNI)
        z = eta - kappa_val
        uNI = -self.controller_gain * z
        self.action = self._NH2ctrl_Cart(xNI, eta, uNI)
        self.action_old = self.action

        return self.action

## Testing part

In [ ]:
state_init = np.array([5, 5, 3 * np.pi / 4, 0, 0])
action_init = np.ones(2)

system = Sys3WRobot(
    dim_input=2,
    dim_output=5,
    sys_type="diff_eqn",
    dim_state=5,
    dim_disturb=5,
    pars=[10, 1]
)

controller = Controller3WRobotDisassembledCLF(
    controller_gain=100, action_bounds=np.array([[-200, 200],[-100, 100]]), time_start=0, sampling_time=0.01, m=10, I=1
)

dashboard = dashboards.RobotTrackingDasboard

scenario, animator = create_testing_pipeline(
    system, 
    controller, 
    state_init=state_init, 
    action_init=action_init, 
    model_of_running_objective = ModelQuadForm(weights=np.diag([5, 5, 1, 0, 0, 0, 0])),
    observation_namings=["x", "y", "angle", "lin. speed", "ang. speed"],
    action_namings=["F [N]", "M [Nm]"],
    system_visualization_dashboard=dashboard,
    system_visualization_dashboard_params={
        "time_start": 0, 
        "x_max": 10, 
        "x_min": -10, 
        "y_max": 10, 
        "y_min": -10
        }
    )

In [ ]:

scenario.run() # <--- takes some time. In TA's solution not more that half the minute
trajectory = []
for i , (k, v) in enumerate(scenario.cache.items()):
    trajectory.append(v[3])

trajectory[-30:]

In [ ]:
%%capture
animation = animator.get_animation()


In [ ]:
animation